In [1]:
import niprov
from dipy.segment.mask import median_otsu
import dipy.reconst.dti as dti
reload(dti)
import numpy as np
import nibabel as nib
from scipy.ndimage.measurements import center_of_mass as com
import dipy.data as dpd
from dipy.segment.mask import applymask
from dipy.core.gradients import gradient_table
from dipy.io import read_bvals_bvecs
from pylabs.utils.paths import getnetworkdataroot
fs = getnetworkdataroot()
from nipype.interfaces.fsl import Eddy
eddy = Eddy(num_threads=24, output_type='NIFTI')
from nipype.interfaces import fsl
flt = fsl.FLIRT(bins=640, interp='nearestneighbour', cost_func='mutualinfo', output_type='NIFTI')
applyxfm = fsl.ApplyXfm(interp='nearestneighbour', output_type='NIFTI')
bet = fsl.BET(output_type='NIFTI')
prov = niprov.ProvenanceContext()

import pylabs, os, inspect
from os.path import split
from os.path import join
pylabs_basepath = split(split(inspect.getabsfile(pylabs))[0])[0]
#set up files to process
project = 'bbc'
subjid = [105]
fname_templ = 'sub-bbc{sid}_ses-{snum}_{meth}_{runnum}'
sespassqc = [1]
methodpassqc = ['dti_15dir_b1000']
runpassqc = [1]
dwi_fnames = [fname_templ.format(sid=str(s), snum=str(ses), meth=m, runnum=str(r)) for s, ses, m, r in zip(subjid, sespassqc, methodpassqc, runpassqc)]
#add for loop over dwi_fnames here:
dwi_fname = dwi_fnames[0]
fpath = join(fs, project, dwi_fname.split('_')[0] , dwi_fname.split('_')[1], 'dwi')
fdwi = join(fpath, dwi_fname + '.nii')
fbvecs = join(fpath, dwi_fname + '.bvecs')
fbvals = join(fpath, dwi_fname + '.bvals')
fdwell = join(fpath, dwi_fname + '.dwell_time')
S0_fname = join(fpath, dwi_fname + '_S0.nii')
#make dipy gtab and load dwi data
bvals, bvecs = read_bvals_bvecs(fbvals, fbvecs)
gtab = gradient_table(bvals, bvecs)
img = nib.load(fdwi)
data = img.get_data()
#make S0 and bet to get mask
S0 = data[:, :, :, gtab.b0s_mask]
nS0_img = nib.Nifti1Image(S0, img.affine)
nS0_img.set_qform(img.affine, code=1)
nib.save(nS0_img, S0_fname)
#make mat file to apply mask and com
flt.inputs.in_file = join(pylabs_basepath, 'data', 'atlases', 'MNI152_T1_1mm_bet_zcut.nii.gz')
flt.inputs.reference = S0_fname
flt.inputs.out_matrix_file = S0_fname[: -6] + 'bet2S0.mat'
flt.inputs.out_file = S0_fname[: -6] + 'S0_zcut.nii'
res = flt.run() 
#apply mat file to center of mass ROI in MNI template
applyxfm.inputs.in_matrix_file = S0_fname[: -6] + 'bet2S0.mat'
applyxfm.inputs.in_file = join(pylabs_basepath, 'data', 'atlases', 'MNI152_T1_1mm-com-mask8k.nii.gz')
applyxfm.inputs.out_file = join(fpath, dwi_fname + '_S0_match_bet_com_roi.nii')
applyxfm.inputs.reference = S0_fname
applyxfm.inputs.apply_xfm = True
result = applyxfm.run()
#apply mat file to MNI mask file to cut off neck
applyxfm.inputs.in_matrix_file = S0_fname[: -6] + 'bet2S0.mat'
applyxfm.inputs.in_file = join(pylabs_basepath, 'data', 'atlases', 'MNI152_T1_1mm_bet_zcut_mask.nii.gz')
applyxfm.inputs.out_file = join(fpath, dwi_fname + '_S0_mask.nii')
applyxfm.inputs.reference = S0_fname
applyxfm.inputs.apply_xfm = True
result = applyxfm.run()
#chop off neck with MNI zcut
zcut_data = nib.load(join(fpath, dwi_fname + '_S0_mask.nii')).get_data()
zcut_data_maskb = zcut_data > 0
S0_mask = np.zeros(np.squeeze(S0).shape)  # need to add a fourth dim here
S0_mask[zcut_data_maskb] = 1
S0_zcut = applymask(S0, S0_mask)
nzcut_img = nib.nifti1.Nifti1Image(S0_zcut, img.affine)
nzcut_img.set_qform(img.affine, code=1)
nib.save(nzcut_img, join(fpath, dwi_fname + '_S0_zcut.nii'))

#get com for fsl bet
com_data = nib.load(join(fpath, dwi_fname + '_S0_match_bet_com_roi.nii')).get_data()
com_data_maskb = com_data > 4000
com_data_mask = np.zeros(com_data.shape)
com_data_mask[com_data_maskb] = 1
match_com = np.round(com(com_data_mask)).astype(int)

#extract brain and make brain mask before eddy current correction
brain_outfname = S0_fname[: -6] + 'S0_brain' 
bet.inputs.in_file = join(fpath, dwi_fname + '_S0_zcut.nii')
bet.inputs.center = list(match_com)
bet.inputs.frac = 0.3
bet.inputs.mask = True
bet.inputs.skull = True
bet.inputs.out_file = brain_outfname + '.nii'
betres = bet.run()

In [2]:
#make index and acquisition parameters files
with open(join(fpath, 'index.txt'), 'w') as f:
    f.write('1 ' * len(gtab.bvals))

with open(fdwell, 'r') as f:
    dwell=f.read().replace('\n', '')

with open(join(fpath, 'acq_params.txt'), 'w') as f:
    f.write('0 1 0 ' + dwell)
#execute addy command in subprocess in local working directory
from pylabs.utils import run_subprocess, WorkingContext
cmd = 'eddy --acqp=acq_params.txt --bvals=' + fbvals + ' --bvecs=' + fbvecs
cmd += ' --imain=' + fdwi + ' --index=index.txt --mask=' + brain_outfname + '_mask.nii '
cmd += '--out=' + fdwi[:-4] + '_eddy_corrected'
with WorkingContext(fpath):
    run_subprocess(cmd)

In [ ]:
#shortcut skips masking and eddy. only does fitting
%matplotlib inline
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from dipy.denoise.noise_estimate import estimate_sigma
from time import time
from dipy.denoise.non_local_means import non_local_means
from dipy.denoise.adaptive_soft_matching import adaptive_soft_matching
from os.path import join
from dipy.core.gradients import gradient_table
from dipy.io import read_bvals_bvecs
import dipy.reconst.dti as dti
from pylabs.utils.paths import getnetworkdataroot
fs = getnetworkdataroot()
#set up files post eddy
project = 'bbc'
subjid = [205]
fname_templ = 'sub-bbc{sid}_ses-{snum}_{meth}_{runnum}'
sespassqc = [1]
methodpassqc = ['dti_15dir_b1000']
runpassqc = [1]
dwi_fnames = [fname_templ.format(sid=str(s), snum=str(ses), meth=m, runnum=str(r)) for s, ses, m, r in zip(subjid, sespassqc, methodpassqc, runpassqc)]
#add for loop over dwi_fnames here:
dwi_fname = dwi_fnames[0]
fpath = join(fs, project, dwi_fname.split('_')[0] , dwi_fname.split('_')[1], 'dwi')
fdwi = join(fpath, dwi_fname + '.nii')
fbvecs = join(fpath, dwi_fname + '.bvecs')
fbvals = join(fpath, dwi_fname + '.bvals')
fbvecs_ec = join(fpath, dwi_fname + '_eddy_corrected.eddy_rotated_bvecs')
S0_fname = join(fpath, dwi_fname + '_S0.nii')
brain_outfname = S0_fname[: -6] + 'S0_brain'
#make gtab
bvals_ec, bvecs_ec = read_bvals_bvecs(fbvals, fbvecs_ec)
gtab_ec = gradient_table(bvals_ec, bvecs_ec)
#open and denoise ec data
img_ec = nib.load(join(fpath, fdwi[:-4] + '_eddy_corrected.nii.gz'))
data_ec = img_ec.get_data()
sigma4 = estimate_sigma(data_ec, N=4)
den4_data_ec = non_local_means(data_ec, sigma=sigma4, mask=mask)
sigma8 = estimate_sigma(data_ec, N=8)
den8_data_ec = non_local_means(data_ec, sigma=sigma8, mask=mask)
#save denoised images
den4_data_ec_img = nib.nifti1.Nifti1Image(den, img.affine)
den4_data_ec_img.set_qform(img_ec.affine, code=1)
nib.save(den4_data_ec_img, join(fpath, dwi_fname + '_ec_den4_dipy.nii'))
den8_data_ec_img = nib.nifti1.Nifti1Image(den, img.affine)
den8_data_ec_img.set_qform(img_ec.affine, code=1)
nib.save(den8_data_ec_img, join(fpath, dwi_fname + '_ec_den8_dipy.nii'))

#fit tensors
tenmodel_wls = dti.TensorModel(gtab_ec, fit_method='WLS')
tenmodel_ols = dti.TensorModel(gtab_ec, fit_method='OLS')
wls_fit = tenmodel_wls.fit(data_ec, mask)
ols_fit = tenmodel_ols.fit(data_ec, mask)



for t in [ec, den4, den8]:
    img_ec = nib.load(join(fpath, fdwi[:-4] + '_eddy_corrected.nii.gz'))
    data_ec = img_ec.get_data()
    mask_img = nib.load(brain_outfname + '_mask.nii')
    mask = mask_img.get_data()
    for m in ['OLS', 'WLS']
        tenmodel = dti.TensorModel(gtab_ec, fit_method=m)
        
    den_img = nib.nifti1.Nifti1Image(den, img.affine, img.header)
    den_img.set_qform(img.affine, code=1)
    nib.save(den_img, join(fpath, dwi_fname + '_ec_den8_dipy.nii'))


    den_tenfit = tenmodel.fit(den)

In [ ]:
#shortcut skips masking and eddy. only does fitting
%matplotlib inline
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from dipy.denoise.noise_estimate import estimate_sigma
from time import time
from dipy.denoise.non_local_means import non_local_means
from dipy.denoise.adaptive_soft_matching import adaptive_soft_matching
from os.path import join
from dipy.core.gradients import gradient_table
from dipy.io import read_bvals_bvecs
from pylabs.utils.paths import getnetworkdataroot
fs = getnetworkdataroot()
#set up files post eddy
project = 'bbc'
subjid = [209]
fname_templ = 'sub-bbc{sid}_ses-{snum}_{meth}_{runnum}'
sespassqc = [1]
methodpassqc = ['dti_15dir_b1000']
runpassqc = [1]
dwi_fnames = [fname_templ.format(sid=str(s), snum=str(ses), meth=m, runnum=str(r)) for s, ses, m, r in zip(subjid, sespassqc, methodpassqc, runpassqc)]
#add for loop over dwi_fnames here:
dwi_fname = dwi_fnames[0]
fpath = join(fs, project, dwi_fname.split('_')[0] , dwi_fname.split('_')[1], 'dwi')
fdwi = join(fpath, dwi_fname + '.nii')
fbvecs = join(fpath, dwi_fname + '.bvecs')
fbvals = join(fpath, dwi_fname + '.bvals')
fbvecs_ec = join(fpath, dwi_fname + '_eddy_corrected.eddy_rotated_bvecs')
S0_fname = join(fpath, dwi_fname + '_S0.nii')
brain_outfname = S0_fname[: -6] + 'S0_brain'

bvals_ec, bvecs_ec = read_bvals_bvecs(fbvals, fbvecs_ec)
gtab_ec = gradient_table(bvals_ec, bvecs_ec)
img = nib.load(fdwi)
img_ec = nib.load(join(fpath, fdwi[:-4] + '_eddy_corrected.nii.gz'))
data_ec = img_ec.get_data()
mask_img = nib.load(brain_outfname + '_mask.nii')
mask = mask_img.get_data()

assert np.allclose(img_ec.affine, img.affine, 5), 'eddy corrected affine does not match orig for ' + join(fpath, fdwi[:-4] + '_eddy_corrected.nii')
assert np.allclose(mask_img.affine, img.affine, 5), 'qform and new affine do not match for ' + brain_outfname + '_mask.nii'
print("vol size", data_ec.shape)
t = time()
sigma = estimate_sigma(data_ec, N=8)
print(data_ec.shape, mask.shape, sigma)
den = non_local_means(data_ec, sigma=sigma, mask=mask)

print("total time", time() - t)
print("vol size", den.shape)

axial_middle = data_ec.shape[2] / 2

before = data_ec[:, :, axial_middle].T
after = den[:, :, axial_middle].T
difference = np.abs(after.astype('f8') - before.astype('f8'))
difference[~mask[:, :, axial_middle].T] = 0

#fig, ax = plt.subplots(1, 3)
#ax[0].imshow(before, cmap='gray', origin='lower')
#ax[0].set_title('before')
#ax[1].imshow(after, cmap='gray', origin='lower')
#ax[1].set_title('after')
#ax[2].imshow(difference, cmap='gray', origin='lower')
#ax[2].set_title('difference')
#for i in range(3):
#    ax[i].set_axis_off()

#plt.show()

In [ ]:
den_img = nib.nifti1.Nifti1Image(den, img.affine, img.header)
den_img.set_qform(img.affine, code=1)
nib.save(den_img, join(fpath, dwi_fname + '_ec_den8_dipy.nii'))

In [ ]:
import dipy.reconst.dti as dti
tenmodel = dti.TensorModel(gtab_ec)
den_tenfit = tenmodel.fit(den)

In [ ]:
#make and save wls FA and MD
FA = den_tenfit.fa
MD = den_tenfit.md
FA_img = nib.nifti1.Nifti1Image(FA, img.affine)
FA_img.set_qform(img.affine, code=1)
nib.save(FA_img, join(fpath,dwi_fname + '_ec_den8_dipy_FA.nii'))
MD_img = nib.nifti1.Nifti1Image(MD, img.affine)
MD_img.set_qform(img.affine, code=1)
nib.save(MD_img, join(fpath, dwi_fname + '_ec_den8_dipy_MD.nii'))

In [ ]:
#make and save ols FA and MDdti_ols = dti.TensorModel(gtab_ec, fit_method="OLS")
den_ols_tenfit = dti_ols.fit(den)
oFA = den_ols_tenfit.fa
oMD = den_ols_tenfit.md
oFA_img = nib.nifti1.Nifti1Image(oFA, img.affine)
oFA_img.set_qform(img.affine, code=1)
nib.save(oFA_img, join(fpath, dwi_fname + '_ec_den8_dipyols_FA.nii'))
oMD_img = nib.nifti1.Nifti1Image(oMD, img.affine)
oMD_img.set_qform(img.affine, code=1)
nib.save(oMD_img, join(fpath, dwi_fname + '_ec_den8_dipyols_MD.nii'))

In [ ]:
#plotting gradient fields
outdir='/media/DiskArray/shared_data/js/bbc/defunct_data/test_dwi_out'
import os
os.chdir(outdir)
from dipy.viz import fvtk
ren = fvtk.ren()
ren.SetBackground(1, 1, 1)
colors_b1000 = fvtk.colors.blue * np.ones(gtab_ec.bvecs.shape)
fvtk.add(ren, fvtk.point(gtab_ec.gradients, colors_b1000, point_radius=100))
print('Saving illustration as bbc209_gradients.png')
fvtk.record(ren, out_path='bbc209_gradients.png', size=(600, 600))
fvtk.show(ren)